# Effectiveness - Add a single point

In [ ]:
import sys
sys.path.append("..")

from sklearn import svm
import numpy as np

import dynashap

from examples.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize
)

In [ ]:
# read
x_train, y_train, x_test, y_test, columns_name = preprocess_data('train_100p_3c.csv', 'test_100p_3c.csv')

# pick a point from test for add
select_idx = 25

add_point_x = x_test[select_idx][:]
add_point_y = y_test[select_idx]

x_train_added = np.append(x_train, [x_test[select_idx,:]], axis=0)
y_train_added = np.append(y_train, y_test[select_idx])

x_test_added = np.delete(x_test, select_idx, axis=0)
y_test_added = np.delete(y_test, select_idx)

x_test = x_test_added
y_test = y_test_added

In [ ]:
#mc plus
model = svm.SVC(decision_function_shape='ovo')

mc_plus_sv_101 = dynashap.mc_shap(x_train_added, y_train_added, x_test_added, y_test_added, model,
                                  1000*101, 25)
save_npy('mc_plus_sv_101.npy', mc_plus_sv_101)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [18, 56, 89]

for idx in idxs:
    save_npy('mc_plus_sv_99_del'+str(idx)+'.npy',
             dynashap.mc_shap(np.delete(x_train, idx, axis=0), np.delete(y_train, idx),
                     x_test_added, y_test_added, model,
                              1000*99, 25)
            )
# mc part
model = svm.SVC(decision_function_shape='ovo')


for i in range(1):
    mc_sv_101 = dynashap.mc_shap(x_train_added, y_train_added, x_test_added, y_test_added, model,
                                 101*100, 25)
    save_npy('mc_sv_101.npy'+str(i), mc_sv_101)
    # mc_sv_101 and mc_sv_101 (many permutations)
    var1 = variance(mc_plus_sv_101[:100], normalize(mc_plus_sv_101[:100], mc_sv_101[:100]))
    print("variance of MC   \t : %.10f" % var1)

# plus version
model = svm.SVC(decision_function_shape='ovo')

pivot_shap = dynashap.PivotShap(x_train, y_train, x_test, y_test, model, init_sv=[])
pivot_shap.prepare(100000, proc_num=25)

pivot_sv_101 = pivot_shap.add_single_point(add_point_x, add_point_y,
                                           101*100)
save_npy('pivot_sv_101_plus.npy', pivot_sv_101)

# init sv
model = svm.SVC(decision_function_shape='ovo')

mc_plus_sv_100 = dynashap.mc_shap(x_train, y_train, x_test_added, y_test_added, model,
                                 1000*len(y_train), 4)
save_npy('mc_plus_sv_100.npy', mc_plus_sv_100)

model = svm.SVC(decision_function_shape='ovo')
mc_sv_101 = dynashap.mc_shap(x_train_added, y_train_added, x_test_added, y_test_added, model,
                           100*len(y_train_added), 1)
save_npy('mc_sv_101.npy', mc_sv_101)

# delta
init_sv = load_npy('mc_plus_sv_100.npy')

delta_shap = dynashap.DeltaShap(x_train, y_train, x_test_added, y_test_added, model,
                                init_sv)
delta_sv_101 = delta_shap.add_single_point(add_point_x, add_point_y,
                            100*(len(y_train)+1))

save_npy('delta_sv_101.npy', delta_sv_101)

# pivot
init_sv = load_npy('mc_plus_sv_100.npy')

pivot_shap = dynashap.PivotShap(x_train, y_train, x_test_added, y_test_added, model,
                              init_sv)

lsv_100 = pivot_shap.prepare(1000*len(y_train))
save_npy('lsv_100.npy', lsv_100)

sv_81 = pivot_shap.add_single_point(add_point_x[0], add_point_y[0],
                            100*(len(y_train)+1), flags={'flag_lsv': True})

pivot_shap.x_train = np.append(x_train, [add_point_x[0]], axis=0)
pivot_shap.y_train = np.append(y_train, add_point_y[0])

pivot_sv_101 = pivot_shap.add_single_point(add_point_x[1], add_point_y[1],
                                        100*(len(y_train)+2), flags={'flag_lsv': True})

save_npy('pivot_sv_101.npy', pivot_sv_101)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_sv = load_npy('mc_plus_sv_100.npy')

# KNN
heur_shap = dynashap.HeurShap(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_sv,
                              params={'method': 'knn'})
heur_shap.prepare(params={'n_neighbors': 3})
knn_sv_101 = heur_shap.add_single_point(add_point_x, add_point_y)
save_npy('knn_sv_101.npy', knn_sv_101)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [18, 56, 89]
svs = list()

for idx in idxs:
    sv = dynashap.mc_shap(np.delete(x_train, idx, axis=0),
                          np.delete(y_train, idx),
                          x_test_added, y_test_added,
                          model, 1000*99, 4)
    save_npy('mc_plus_sv_99_del'+str(idx)+'.npy', sv)
    svs.append(sv)

svs = np.asarray(svs)

save_npy('knn_plus_svs.npy', svs)


# KNN+
heur_shap = dynashap.HeurShap(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_sv,
                              params={'method': 'knn+'})

heur_shap.prepare(flags={'exact': False, 'train': False},
                  params={'n_neighbors': 3,
                          'simi_type': 'ed',
                          'f_shap': 'n*n',
                          'rela': ['poly', 1],
                          'train_idxs': [18, 56, 89],
                          'train_svs': load_npy('knn_plus_svs.npy')})

knn_plus_sv_101 = heur_shap.add_single_point(add_point_x, add_point_y)
save_npy('knn_plus_sv_101.npy', knn_plus_sv_101)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_sv = load_npy('mc_plus_sv_100.npy')
base_shap = dynashap.BaseShap(x_train, y_train, x_test_added,
                              y_test_added, model, init_sv)

# average based
base_sv_101_avg = base_shap.add_single_point(add_point_x,
                                            add_point_y,
                                            params={'method': 'avg'})

# leave one out based
base_sv_101_loo = base_shap.add_single_point(add_point_x,
                                            add_point_y,
                                            params={'method': 'loo'})

save_npy('base_sv_101_avg.npy', base_sv_101_avg)

save_npy('base_sv_101_loo.npy', base_sv_101_loo)

# comparison
def comp(base_v, comp_sv, name):
    var = variance(base_v, normalize(base_v, comp_sv))
    print("variance of %s   \t : %.10f" % (name, var))

mc_plus_sv_101 = load_npy('mc_plus_sv_101.npy')

comp(mc_plus_sv_101, load_npy('delta_sv_101.npy'), 'delta')
comp(mc_plus_sv_101, load_npy('pivot_sv_101.npy'), 'pivot')
comp(mc_plus_sv_101, load_npy('knn_sv_101.npy'), 'knn')
comp(mc_plus_sv_101, load_npy('knn_plus_sv_101.npy'), 'knn+')
comp(mc_plus_sv_101, load_npy('mc_sv_101.npy'), 'mc')
comp(mc_plus_sv_101, load_npy('base_sv_101_loo.npy'), 'base loo')
comp(mc_plus_sv_101, load_npy('base_sv_101_avg.npy'), 'base avg')